In [3]:
!pip install torchinfo

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import os
#set workdir
os.chdir("/content/gdrive/My Drive/Projects/MSc-Project-River-Segmentation/MSc-Project/Code")

In [ ]:
! ls


config.cfg  dataloader.py  models  __pycache__	Testing.ipynb  Training.ipynb


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from dataloader import Dataset
from torchinfo import summary
import time
import copy

from tqdm import tqdm

from models.unet import UNet
from models.simple import Simple

from dataloader import Dataset

In [ ]:
PARAMS = {
    "input_size": 416,
    "model": "vgg_unet",
    "learning_rate": 0.001,
    
}

In [7]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Training.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Evaluation.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [8]:
!git add .

In [10]:
!git commit -m "updated"
!git config --global user.email "moad.s.k@gmail.com"
!git config --global user.name "MoadSaadKhorchef"

[main 6fb3f2f] updated
 2 files changed, 2 insertions(+), 1 deletion(-)
 create mode 100644 Code/Evaluation.ipynb


In [11]:
!git push origin main

Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 2.50 KiB | 855.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/MoadSaadKhorchef/MSc-Project.git
   c4792cf..6fb3f2f  main -> main
